# Hungarian algorithm for BP pairings
_Chuan-Zheng Lee <czlee@stanford.edu>_

This notebook does a very simple trial of using the Hungarian algorithm to generate pairings for a BP tournament.

The traditional Hungarian algorithm doesn't try to choose randomly; it simply chooses whichever optimal solution it happens to come across first. There are two ideas to "randomise" the Hungarian algorithm in this notebook:

1. Shuffle the rows and columns of the cost matrix first, then run the Hungarian algorithm on this shuffled matrix
2. Pass in the cost matrix as-is, but during the algorithm, in stages where we are looking for the first row or column, search throw the rows or columns in random order.

We try this for two situations, both involving an eight-team bracket:

1. A "BP-like" cost matrix, which is generated assuming that teams have so far had three rounds, and intentionally ensuring that the same positions in both rooms are of the same cost.
2. A Bernoulli i.i.d. cost matrix, with parameter p = 2/3.

It doesn't appear that the two methods of randomisation are any different to each other.

In [1]:
import random
from collections import Counter
from munkres import Munkres
niterations = 16000

# Check if this is the modified randomized version of the Munkres library,
# available at https://github.com/czlee/munkres/.
try:
    Munkres(random=True)
except TypeError:
    randomized_munkres = False
else:
    randomized_munkres = True

In [2]:
def hungarian_shuffle_first(costs):
    """Applies the Hungarian algorithm to `costs`, but permutes the rows and
    columns of the matrix first. Returns a list of column numbers."""
    n = len(costs)
    I = random.sample(range(n), n)
    J = random.sample(range(n), n)
    C = [[costs[i][j] for j in J] for i in I]
    m = Munkres()
    indices = m.compute(C)
    cols = [0] * n
    for i, j in indices:
        cols[I[i]] = J[j]
    return cols

def hungarian_shuffle_during(costs):
    """Applies a randomized version of the Hungarian algorithm.
    This only works with a modified version of the Munkres library, which
    can be cloned from https://github.com/czlee/munkres/.
    Randomizing during the algorithm seemed to have much the same effect as
    shuffling before it, so won't be pushing these changes into a pull
    request."""
    m = Munkres(random=True)
    indices = m.compute(costs)
    cols = [0] * len(costs)
    for i, j in indices:
        cols[i] = j
    return cols

def total_cost(costs, cols):
    """Computes the total cost according to `costs`, of the chosen columns
    `cols`."""
    rows = range(len(costs))
    return sum(costs[i][j] for i, j in zip(rows, cols))

def generate_bp_costs(d, k):
    """Generates an `4d`-by-`4d` cost matrix that assumes there have been
    `k` debates before now. (`d` represents the number of BP debates.)"""
    costs = []
    for i in range(4 * d):
        history = [random.choice(range(4)) for i in range(k)]
        counts = [history.count(pos) for pos in range(4)]
        row = counts * d  # replicate list `d` times
        costs.append(row)
    return costs
        
def run_simulations(costs, niterations, allocate):
    """Runs the Munkres-shuffle-first algorithm on the matrix `costs`
    `niterations` times, and returns a dict mapping solutions to
    frequencies (how many iterations that solution was obtained)."""
    results = Counter()
    for i in range(niterations):
        results[tuple(allocate(costs))] += 1
    return results

def verify_costs(costs, solutions):
    """Verifies that every key in solutions gives the same cost, and
    returns that cost if so."""
    total_costs = [total_cost(costs, solution) for solution in solutions]
    assert len(set(total_costs)) == 1
    return total_costs[0]

def frequencies_by_position(results):
    """Converts a sequence of solutions into just the positions (0 to 3,
    representing the 4 BP positions), and returns a dict mapping each such
    sequence to a list of frequencies for solutions that reduce to that 
    sequence of positions. Useful only for results of cost matrices generated 
    by `generate_bp_costs`."""
    reduced = {}
    for solution, frequency in results.items():
        positions = tuple(x % 4 for x in solution)
        frequencies = reduced.setdefault(positions, [])
        frequencies.append(frequency)
    return reduced

## British Parliamentary-like simulation

In [3]:
costs = generate_bp_costs(2, 3)
costs

[[0, 2, 1, 0, 0, 2, 1, 0],
 [2, 1, 0, 0, 2, 1, 0, 0],
 [2, 0, 1, 0, 2, 0, 1, 0],
 [0, 2, 1, 0, 0, 2, 1, 0],
 [1, 1, 1, 0, 1, 1, 1, 0],
 [2, 0, 0, 1, 2, 0, 0, 1],
 [1, 2, 0, 0, 1, 2, 0, 0],
 [2, 0, 1, 0, 2, 0, 1, 0]]

In [4]:
results = run_simulations(costs, niterations, hungarian_shuffle_first)
results

Counter({(0, 2, 1, 4, 3, 5, 6, 7): 255,
         (0, 2, 1, 4, 3, 6, 7, 5): 248,
         (0, 2, 1, 4, 7, 5, 6, 3): 260,
         (0, 2, 1, 4, 7, 6, 3, 5): 259,
         (0, 2, 3, 4, 7, 1, 6, 5): 261,
         (0, 2, 3, 4, 7, 5, 6, 1): 238,
         (0, 2, 5, 4, 3, 1, 6, 7): 267,
         (0, 2, 5, 4, 3, 6, 7, 1): 258,
         (0, 2, 5, 4, 7, 1, 6, 3): 251,
         (0, 2, 5, 4, 7, 6, 3, 1): 258,
         (0, 2, 7, 4, 3, 1, 6, 5): 266,
         (0, 2, 7, 4, 3, 5, 6, 1): 230,
         (0, 3, 1, 4, 7, 2, 6, 5): 271,
         (0, 3, 1, 4, 7, 6, 2, 5): 261,
         (0, 3, 5, 4, 7, 2, 6, 1): 252,
         (0, 3, 5, 4, 7, 6, 2, 1): 249,
         (0, 6, 1, 4, 3, 2, 7, 5): 230,
         (0, 6, 1, 4, 3, 5, 2, 7): 247,
         (0, 6, 1, 4, 7, 2, 3, 5): 269,
         (0, 6, 1, 4, 7, 5, 2, 3): 220,
         (0, 6, 3, 4, 7, 1, 2, 5): 232,
         (0, 6, 3, 4, 7, 5, 2, 1): 237,
         (0, 6, 5, 4, 3, 1, 2, 7): 251,
         (0, 6, 5, 4, 3, 2, 7, 1): 242,
         (0, 6, 5, 4, 7, 1, 2, 3): 256,


In [5]:
frequencies_by_position(results)

{(0, 2, 1, 0, 3, 1, 2, 3): [236,
  237,
  255,
  251,
  267,
  265,
  260,
  247,
  255,
  267,
  251,
  237,
  256,
  220,
  253,
  254],
 (0, 2, 1, 0, 3, 2, 3, 1): [267,
  266,
  230,
  218,
  269,
  266,
  259,
  240,
  258,
  248,
  242,
  248,
  248,
  253,
  258,
  235],
 (0, 2, 3, 0, 3, 1, 2, 1): [266,
  261,
  238,
  232,
  246,
  237,
  230,
  241,
  264,
  247,
  261,
  245,
  234,
  253,
  266,
  258],
 (0, 3, 1, 0, 3, 2, 2, 1): [261,
  241,
  246,
  251,
  228,
  236,
  242,
  247,
  272,
  252,
  271,
  236,
  247,
  249,
  272,
  254]}

In [6]:
verify_costs(costs, results)

0

In [7]:
# Only run the next three cells if the randomized version of munkres is available
results = run_simulations(costs, niterations, hungarian_shuffle_during) if randomized_munkres else None

In [8]:
frequencies_by_position(results) if randomized_munkres else None

{(0, 2, 1, 0, 3, 1, 2, 3): [242,
  247,
  245,
  234,
  243,
  247,
  259,
  251,
  255,
  254,
  273,
  254,
  252,
  253,
  229,
  272],
 (0, 2, 1, 0, 3, 2, 3, 1): [244,
  259,
  247,
  261,
  222,
  256,
  241,
  257,
  260,
  245,
  248,
  239,
  248,
  250,
  245,
  223],
 (0, 2, 3, 0, 3, 1, 2, 1): [219,
  247,
  223,
  260,
  274,
  227,
  262,
  257,
  234,
  259,
  234,
  260,
  270,
  259,
  240,
  236],
 (0, 3, 1, 0, 3, 2, 2, 1): [281,
  256,
  208,
  270,
  259,
  258,
  245,
  251,
  277,
  247,
  263,
  251,
  272,
  226,
  264,
  256]}

In [9]:
verify_costs(costs, results) if randomized_munkres else None

0

## Random (Bernoulli i.i.d.) costs

In [10]:
def generate_random_costs(n, p):
    """Generates an `n`-by-`n` cost matrix in which each elements is
    i.i.d. Bernoulli(p). This is probably not a realistic cost matrix
    for a BP bracket."""
    return [[int(random.random() < p) for i in range(n)] for j in range(n)]

In [11]:
costs = generate_random_costs(8, 2/3)
costs

[[1, 1, 0, 1, 1, 1, 0, 1],
 [1, 1, 0, 1, 0, 1, 1, 0],
 [0, 1, 0, 1, 0, 1, 1, 1],
 [1, 1, 1, 1, 1, 0, 1, 0],
 [0, 1, 1, 0, 1, 1, 1, 0],
 [1, 0, 0, 1, 1, 0, 1, 0],
 [0, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 0, 1, 1, 0, 0]]

In [12]:
results = run_simulations(costs, niterations, hungarian_shuffle_first)
results

Counter({(2, 4, 0, 5, 3, 1, 7, 6): 1675,
         (2, 7, 4, 5, 3, 1, 0, 6): 1622,
         (6, 2, 4, 5, 0, 1, 7, 3): 1331,
         (6, 2, 4, 5, 3, 1, 0, 7): 726,
         (6, 2, 4, 5, 3, 7, 0, 1): 682,
         (6, 2, 4, 5, 7, 1, 0, 3): 967,
         (6, 2, 4, 7, 3, 5, 0, 1): 1388,
         (6, 4, 0, 5, 3, 2, 7, 1): 1415,
         (6, 4, 2, 5, 0, 1, 7, 3): 1237,
         (6, 4, 2, 5, 3, 1, 0, 7): 777,
         (6, 4, 2, 5, 3, 7, 0, 1): 756,
         (6, 4, 2, 5, 7, 1, 0, 3): 1035,
         (6, 4, 2, 7, 3, 5, 0, 1): 1454,
         (6, 7, 4, 5, 3, 2, 0, 1): 935})

In [13]:
verify_costs(costs, results)

0

In [14]:
# Only run the next two cells if the randomized version of munkres is available
results = run_simulations(costs, niterations, hungarian_shuffle_during) if randomized_munkres else None
results

Counter({(2, 4, 0, 5, 3, 1, 7, 6): 1792,
         (2, 7, 4, 5, 3, 1, 0, 6): 1578,
         (6, 2, 4, 5, 0, 1, 7, 3): 1320,
         (6, 2, 4, 5, 3, 1, 0, 7): 726,
         (6, 2, 4, 5, 3, 7, 0, 1): 731,
         (6, 2, 4, 5, 7, 1, 0, 3): 894,
         (6, 2, 4, 7, 3, 5, 0, 1): 1293,
         (6, 4, 0, 5, 3, 2, 7, 1): 1417,
         (6, 4, 2, 5, 0, 1, 7, 3): 1193,
         (6, 4, 2, 5, 3, 1, 0, 7): 802,
         (6, 4, 2, 5, 3, 7, 0, 1): 795,
         (6, 4, 2, 5, 7, 1, 0, 3): 959,
         (6, 4, 2, 7, 3, 5, 0, 1): 1421,
         (6, 7, 4, 5, 3, 2, 0, 1): 1079})

In [15]:
verify_costs(costs, results) if randomized_munkres else None

0